# Import Assignments From a CSV File¶
In this example, a CSV file containing the locations of potholes will be imported into a Workforce Project as new assignments.

### Import ArcGIS API for Python
Import the `arcgis` library and some modules within it.

In [ ]:
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce
from arcgis.geocoding import geocode


### Connect to Organization And Get The Project
Let's connect to ArcGIS Online and find the new Project to add assignments to.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("c765482bd0b9479b9104368da54df90d")
project = workforce.Project(item)


### Load the CSV File¶
Let's use the pandas library to read the CSV file and display the potholes.

In [ ]:
df = pd.read_csv("assignments.csv")
df

### Create An Assignment For Each Row
For each assignment, First geocode the address to get the x,y location in (WGS84 Web Mercator) of the assignment. Then supply additional attributes.

Finally use the batch_add method to add multiple assignments at once (this is faster than using the add method since validation is performed once for all assignments).

In [ ]:
assignments = []
for index, row in df.iterrows():
    geometry = geocode(f"{row['Location']}", out_sr=3857)[0]["location"]
    assignments.append(
        workforce.Assignment(
            project,
            geometry=geometry,
            location=row["Location"],
            description=row["Description"],
            priority=row["Priority"],
            work_order_id=row["Work Order Id"],
            assignment_type="Fill in Pothole",
            status="unassigned"
        )
    )
project.assignments.batch_add(assignments)


### Verify the assignments on the map
Let's verify that the assignments were created.

In [ ]:
webmap = gis.map("Palm Springs", zoomlevel=14)
webmap.add_layer(project.assignments_layer)
webmap